In [5]:
import os

# Root dataset directory (update if needed)
dataset_dir = r"D:\SkinCure\myproject\dataset"

# KEEP VARIABLE NAMES SAME (pipeline safe)
train_dir = os.path.join(dataset_dir, "Train")
val_dir   = os.path.join(dataset_dir, "Test")

print("Train Dir:", train_dir)
print("Validation Dir:", val_dir)


Train Dir: D:\SkinCure\myproject\dataset\Train
Validation Dir: D:\SkinCure\myproject\dataset\Test


In [6]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.utils import class_weight


In [7]:
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.2,
    zoom_range=0.25,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)


In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
print("Class Indices:", train_generator.class_indices)

# THIS is what Django must use
class_names = {v: k for k, v in train_generator.class_indices.items()}


Found 13898 images belonging to 22 classes.
Found 1546 images belonging to 22 classes.
Class Indices: {'Acne': 0, 'Actinic_Keratosis': 1, 'Benign_tumors': 2, 'Bullous': 3, 'Candidiasis': 4, 'DrugEruption': 5, 'Eczema': 6, 'Infestations_Bites': 7, 'Lichen': 8, 'Lupus': 9, 'Moles': 10, 'Psoriasis': 11, 'Rosacea': 12, 'Seborrh_Keratoses': 13, 'SkinCancer': 14, 'Sun_Sunlight_Damage': 15, 'Tinea': 16, 'Unknown_Normal': 17, 'Vascular_Tumors': 18, 'Vasculitis': 19, 'Vitiligo': 20, 'Warts': 21}


In [9]:
labels = train_generator.classes

class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


Class Weights: {0: np.float64(1.0653073739077112), 1: np.float64(0.844555177442878), 2: np.float64(0.577975546868502), 3: np.float64(1.2534271284271283), 4: np.float64(2.5472873900293256), 5: np.float64(1.1548944656805717), 6: np.float64(0.6254725472547255), 7: np.float64(1.2055863983344899), 8: np.float64(1.1423639651487754), 9: np.float64(2.031277404267758), 10: np.float64(1.7499370435658523), 11: np.float64(0.7703991130820399), 12: np.float64(2.4871152469577664), 13: np.float64(1.3884115884115884), 14: np.float64(0.9115833661288206), 15: np.float64(2.0247668997668997), 16: np.float64(0.6844282478085295), 17: np.float64(0.3826331149165795), 18: np.float64(1.1634019755566718), 19: np.float64(1.3703411555906133), 20: np.float64(0.8847720906544436), 21: np.float64(1.0891849529780564)}


In [10]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False


In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)


In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,126,102 (92.03 MB)

 Trainable params: 534,294 (2.04 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [13]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3),
    ModelCheckpoint(
        "model.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]


In [14]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 1/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1370 - loss: 3.5691

435/435 ━━━━━━━━━━━━━━━━━━━━ 1285s 3s/step - accuracy: 0.1973 - loss: 3.1542 - val_accuracy: 0.3661 - val_loss: 2.0943 - learning_rate: 1.0000e-04
Epoch 2/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2968 - loss: 2.5754

435/435 ━━━━━━━━━━━━━━━━━━━━ 1197s 3s/step - accuracy: 0.3052 - loss: 2.5308 - val_accuracy: 0.4088 - val_loss: 1.9057 - learning_rate: 1.0000e-04
Epoch 3/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3455 - loss: 2.3157

435/435 ━━━━━━━━━━━━━━━━━━━━ 995s 2s/step - accuracy: 0.3467 - loss: 2.3163 - val_accuracy: 0.4495 - val_loss: 1.7958 - learning_rate: 1.0000e-04
Epoch 4/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3748 - loss: 2.1756

435/435 ━━━━━━━━━━━━━━━━━━━━ 948s 2s/step - accuracy: 0.3760 - loss: 2.1785 - val_accuracy: 0.4541 - val_loss: 1.7490 - learning_rate: 1.0000e-04
Epoch 5/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4029 - loss: 2.0757

...

435/435 ━━━━━━━━━━━━━━━━━━━━ 964s 2s/step - accuracy: 0.5262 - loss: 1.5494 - val_accuracy: 0.5349 - val_loss: 1.4723 - learning_rate: 1.0000e-04
Epoch 20/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5418 - loss: 1.5067

435/435 ━━━━━━━━━━━━━━━━━━━━ 957s 2s/step - accuracy: 0.5331 - loss: 1.5258 - val_accuracy: 0.5433 - val_loss: 1.4644 - learning_rate: 1.0000e-04
Epoch 21/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5409 - loss: 1.5124

435/435 ━━━━━━━━━━━━━━━━━━━━ 986s 2s/step - accuracy: 0.5380 - loss: 1.4968 - val_accuracy: 0.5453 - val_loss: 1.4584 - learning_rate: 1.0000e-04
Epoch 22/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5259 - loss: 1.5141

435/435 ━━━━━━━━━━━━━━━━━━━━ 1336s 3s/step - accuracy: 0.5342 - loss: 1.4907 - val_accuracy: 0.5466 - val_loss: 1.4689 - learning_rate: 1.0000e-04
Epoch 23/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 1134s 3s/step - accuracy: 0.5396 - loss: 1.4795 - val_accuracy: 0.5446 - val_loss: 1.4513 - learning_rate: 1.0000e-04
Epoch 24/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 1135s 3s/step - accuracy: 0.5444 - loss: 1.4656 - val_accuracy: 0.5459 - val_loss: 1.4553 - learning_rate: 1.0000e-04
Epoch 25/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5612 - loss: 1.4015

435/435 ━━━━━━━━━━━━━━━━━━━━ 1098s 3s/step - accuracy: 0.5503 - loss: 1.4414 - val_accuracy: 0.5524 - val_loss: 1.4584 - learning_rate: 1.0000e-04


In [15]:
base_model.trainable = True

for layer in base_model.layers[:120]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 73s/step - accuracy: 0.5054 - loss: 1.5942  

435/435 ━━━━━━━━━━━━━━━━━━━━ 31816s 73s/step - accuracy: 0.5088 - loss: 1.5718 - val_accuracy: 0.5640 - val_loss: 1.4683 - learning_rate: 1.0000e-05
Epoch 2/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5597 - loss: 1.4152

435/435 ━━━━━━━━━━━━━━━━━━━━ 1941s 4s/step - accuracy: 0.5614 - loss: 1.4047 - val_accuracy: 0.5776 - val_loss: 1.4263 - learning_rate: 1.0000e-05
Epoch 3/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5669 - loss: 1.3814

435/435 ━━━━━━━━━━━━━━━━━━━━ 2121s 5s/step - accuracy: 0.5734 - loss: 1.3561 - val_accuracy: 0.5854 - val_loss: 1.3932 - learning_rate: 1.0000e-05
Epoch 4/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5865 - loss: 1.2887

435/435 ━━━━━━━━━━━━━━━━━━━━ 1976s 5s/step - accuracy: 0.5914 - loss: 1.2806 - val_accuracy: 0.5918 - val_loss: 1.3740 - learning_rate: 1.0000e-05
Epoch 5/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6050 - loss: 1.2430

435/435 ━━━━━━━━━━━━━━━━━━━━ 2062s 5s/step - accuracy: 0.6122 - loss: 1.2283 - val_accuracy: 0.6022 - val_loss: 1.3505 - learning_rate: 1.0000e-05
Epoch 6/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6345 - loss: 1.1772

435/435 ━━━━━━━━━━━━━━━━━━━━ 1624s 4s/step - accuracy: 0.6323 - loss: 1.1668 - val_accuracy: 0.6138 - val_loss: 1.3411 - learning_rate: 1.0000e-05
Epoch 7/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 1956s 4s/step - accuracy: 0.6444 - loss: 1.1291 - val_accuracy: 0.6119 - val_loss: 1.3328 - learning_rate: 1.0000e-05
Epoch 8/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 2000s 5s/step - accuracy: 0.6588 - loss: 1.0733 - val_accuracy: 0.6138 - val_loss: 1.3387 - learning_rate: 1.0000e-05
Epoch 9/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6727 - loss: 1.0466

435/435 ━━━━━━━━━━━━━━━━━━━━ 2027s 5s/step - accuracy: 0.6688 - loss: 1.0478 - val_accuracy: 0.6197 - val_loss: 1.3061 - learning_rate: 1.0000e-05
Epoch 10/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.6837 - loss: 1.0039

435/435 ━━━━━━━━━━━━━━━━━━━━ 3756s 9s/step - accuracy: 0.6838 - loss: 0.9997 - val_accuracy: 0.6300 - val_loss: 1.3075 - learning_rate: 1.0000e-05


In [16]:
loss, acc = model.evaluate(val_generator)
print("Final Validation Accuracy:", acc)


49/49 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.6197 - loss: 1.3061
Final Validation Accuracy: 0.619663655757904


In [17]:
model.save("model.h5")
print("✅ model.h5 saved successfully")


✅ model.h5 saved successfully


In [18]:
import json

with open("class_mapping.json", "w") as f:
    json.dump(class_names, f)

print("✅ class_mapping.json saved")


✅ class_mapping.json saved
